In [290]:
#####Latent Dirichlet Allocation Model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

####データの発生####
##データの設定
k = 15   #トピック数
d = 2000   #文書数
v  = 500   #語彙数
w = poisson(175, d)   #1文書あたりの単語数
f = sum(w)   #総単語数

In [291]:
##パラメータの設定
#ディレクリ分布のパラメータの設定
alpha0 = np.repeat(0.2, k)   #文書のディレリ事前分布のパラメータ
alpha1 = np.repeat(0.15, v)   #単語のディレクリ事前分布のパラメータ

#ディレクリ分布からパラメータを発生
thetat = theta = np.random.dirichlet(alpha0, d)   #文書のトピック分布をディレクリ乱数から発生
phit = phi = np.random.dirichlet(alpha1, k)    #単語のトピック分布をデレクレリ乱数から発生

In [292]:
##多項分布より単語頻度行列を生成
WX = np.zeros((d, v), dtype='int')
Z = [i for i in range(d)]
vec = np.arange(0, k)

for i in range(d):
    #文書のトピックを生成
    z = multinomial(1, theta[i, :], w[i])   #文書のトピック分布を発生
    z_vec = np.dot(z, vec)

    #トピック割当から単語を生成
    word = np.zeros((w[i], v))
    for j in range(w[i]):
        word[j, :] = multinomial(1, phi[z_vec[j], :], 1)

    WX[i, :] = np.sum(word, axis=0)
    Z[i] = z_vec

In [293]:
####トピックモデルのためのデータと関数の準備####
##それぞれの文書中の単語の出現をベクトルに並べる
##データ推定用IDを作成
ID_list = [i for i in range(d)]
wd_list = [i for i in range(d)]

#文書ごとに求人IDおよび単語IDを作成
for i in range(d):
    ID_list[i] = np.repeat(i, w[i])
    num1 = (WX[i, :] > 0) * np.arange(1, v+1)
    num2 = num1[num1!=0]
    W1 = WX[i, WX[i, :] > 0]
    number = np.repeat(num2, W1)
    wd_list[i] = number

#リストをベクトル変換
wd = np.zeros(sum(w), dtype='int')
ID_d = np.zeros(sum(w), dtype='int')
start = 0

for i in range(d):
    wd[start:start+w[i]] = wd_list[i] - 1
    ID_d[start:start+w[i]] = ID_list[i]
    start += w[i] 

In [294]:
##インデックスを作成
doc_list = [i for i in range(d)]
doc_vec = [i for i in range(d)]
word_list = [i for i in range(v)]
word_vec = [i for i in range(v)]
index = np.array(range(f))

for i in range(d):
    doc_list[i] = index[ID_d==i]
    doc_vec[i] = np.repeat(1, doc_list[i].shape)
for i in range(v):
    word_list[i] = index[wd==i]
    word_vec[i] = np.repeat(1, word_list[i].shape)

In [295]:
####マルコフ連鎖モンテカルロ法でLDAのパラメータをサンプリング####
##単語ごとに尤度と負担率を計算する関数
def burden_fr(f, theta, phi, wd, w, k):
    Bur = np.zeros((f, k))
    Bur0 = np.zeros((f, k))

    #負担係数を計算
    for j in range(k):
        Bi = np.repeat(theta[:, j], w) * phi[j, wd]
        Bur[:, j] = Bi

    #負担率の分母部分
    Bur_sums = np.sum(Bur, axis=1)
    for l in range(k):
        Bur0[:, l] = Bur_sums 

    #負担率と混合率を計算
    Br = Bur / Bur0   #負担率の計算
    r = np.sum(Br, axis=0) / np.sum(Br)   #混合率の計算
    return Br, Bur, r

In [310]:
##アルゴリズムの設定
R = 2000
keep = 2
burnin = 500
iter = 0 
disp =10

In [311]:
##事前分布の設定
alpha1 = 1.0 
beta1 = 0.5

In [312]:
##パラメータの初期値
mu = np.sum(WX, axis=0)/np.sum(WX)
phi = np.random.dirichlet(np.repeat(1, v), k)
theta = np.random.dirichlet(np.repeat(1, k), d)

In [313]:
##パラメータの保存用配列
THETA = np.zeros((d, k, R//keep), dtype='int')
PHI = np.zeros((k, v, R//keep), dtype='int')
SEG = np.zeros((f, k), dtype='int')

In [314]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##単語トピックを生成
    #潜在確率を計算
    out = burden_fr(f, theta, phi, wd, w, k)
    Br = out[0]
    r = out[2]

    #多項分布よりトピックを生成
    Zi = np.zeros((f, k), dtype="int")
    for i in range(f):
       Zi[i, :] = multinomial(1, Br[i, :], 1)
    zi_vec = np.dot(Zi, np.arange(1, k+1))
    Zi_T = Zi.T


    ##パラメータを更新
    #トピック分布thetaをサンプリング
    wsum = np.zeros((d, k))
    for i in range(d):
        wsum[i :] = np.dot(Zi_T[:, doc_list[i]], doc_vec[i]) + alpha1   #ディクレリ分布のパラメータ
        theta[i, :] = np.random.dirichlet(wsum[i, :], 1)

    #単語分布phiをサンプリング
    vsum = np.zeros((k, v))
    for j in range(v):
        vsum[:, j] = np.dot(Zi_T[:, word_list[j]], word_vec[j]) + beta1   #ディクレリ分布のパラメータ
    for i in range(k):
        phi[i, :] = np.random.dirichlet(vsum[i, :], 1)
        
    ##サンプリング結果の格納とパラメータの表示
    #サンプリング結果の保存
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = theta
        PHI[:, :, mkeep] = phi
        
        #バーンインを超えたらトピックを格納
        if rp >= burnin:
            SEG += Zi  
    
        #パラメータの表示
        if rp%disp==0:
            LL = np.sum(np.log(np.sum(out[1], axis=1)))   #対数尤度を計算
            print(rp)
            print(LL)

0
-2200006.96106
10
-2087580.16367
20
-2027396.8021
30
-1997274.42955
40
-1982585.70648
50
-1979068.04032
60
-1978629.28508
70
-1978626.32535
80
-1978349.34576
90
-1978353.33705
100
-1977905.20144
110
-1974508.98921
120
-1970226.89627
130
-1969701.42805
140
-1969627.37155
150
-1969532.11779
160
-1969511.73877
170
-1969511.86411
180
-1969452.19635
190
-1969560.82504
200
-1969613.34582
210
-1969811.22622
220
-1969920.11823
230
-1969450.8505
240
-1969757.85799
250
-1969553.10945
260
-1969730.37962
270
-1969872.23529
280
-1969784.62625
290
-1969605.89542
300
-1969854.07765
310
-1969515.0274
320
-1969828.33866
330
-1969451.37219
340
-1969690.18635
350
-1969687.99829
360
-1969780.31654
370
-1969672.87958
380
-1969728.16478
390
-1969546.24066
400
-1969689.48095
410
-1969679.57329
420
-1969918.22916
430
-1969332.2894
440
-1969889.26077
450
-1969658.76743
460
-1969608.61182
470
-1969617.86604
480
-1969542.41707
490
-1969828.65868
500
-1969619.93613
510
-1969867.18144
520
-1969819.78304
530
-196

KeyboardInterrupt: 